In [1]:
import pandas as pd
import numpy as np
import psycopg2 as ps

In [2]:
df_ciudad = pd.read_csv('../data/ciudad.csv')
df_eventos = pd.read_csv("../data/eventos.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
df_hoteles = pd.read_csv('../data/hoteles.csv')
df_clientes = pd.read_csv('../data/clientes.csv')
df_reservas = pd.read_csv("../data/reservas.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'])

In [ ]:
df_eventos.duplicated().sum()

In [4]:
df_ciudad = df_ciudad.drop_duplicates()
df_hoteles = df_hoteles.drop_duplicates()
df_clientes = df_clientes.drop_duplicates()

In [ ]:
df_ciudad.duplicated().sum()

In [27]:
conn = ps.connect(
    dbname="proyecto_etl",
    user="my_user",
    password="admin",
    host="localhost",
    port="5432"
)

In [28]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT version();")
cur.fetchone()

In [9]:
# Convertir DataFrame a lista de tuplas
#records = df.to_records(index=False)
#records_list = list(records)

In [ ]:
df_ciudad.info()

In [11]:
d_insert_ciudad = ['Madrid']

In [12]:
insert_query = """
INSERT INTO ciudad (nombre_ciudad) VALUES (%s)
"""

In [13]:
cur.execute(insert_query, d_insert_ciudad)
conn.commit()

In [ ]:
cur.execute('SELECT nombre_ciudad, id_ciudad FROM ciudad')
ciudades = dict(cur.fetchall())
ciudades

In [ ]:
df_eventos.head(1)

In [16]:
d_insert_eventos = []

In [17]:
for i, row in df_eventos.iterrows():
    id_evento = row['id_evento']
    nombre_evento = row['nombre_evento']
    url_evento = row['url_evento']
    codigo_postal = row['codigo_postal'] if row['codigo_postal'] !=0 else None
    direccion = row['direccion'] if pd.notna(row['direccion']) else None
    horario = row['horario'] if pd.notna(row['horario']) else None
    fecha_inicio = row['fecha_inicio']
    fecha_fin = row['fecha_fin']
    organizacion = row['organizacion'] if pd.notna(row['organizacion']) else None
    id_ciudad = ciudades.get('Madrid')

    d_insert_eventos.append((id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))


In [ ]:
d_insert_eventos

In [19]:

query_eventos = """
    INSERT INTO eventos (id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
cur.executemany(query_eventos, d_insert_eventos)
conn.commit()

In [ ]:
df_hoteles = df_hoteles.rename(columns={"id_hotel_final": "id_hotel"})
df_hoteles['id_hotel'] = [(i + 1) for i in range(len(df_hoteles))]
df_hoteles

In [12]:
d_insert_hoteles = []

In [ ]:
for i, row in df_hoteles.iterrows():
    id_hotel = row['id_hotel']
    nombre_hotel = row['nombre_hotel']
    competencia = row['competencia']
    valoracion = row['valoracion']
    id_ciudad = ciudades.get('Madrid')

    d_insert_hoteles.append((id_hotel, nombre_hotel, competencia, valoracion, id_ciudad))

In [16]:

query_hoteles = """
    INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s)
"""

In [17]:
cur.executemany(query_hoteles, d_insert_hoteles)
conn.commit()

In [18]:
d_insert_clientes = []

In [19]:
for i, row in df_clientes.iterrows():
    id_cliente = row['id_cliente']
    nombre = row['nombre']
    apellido = row['apellido']
    mail = row['mail']

    d_insert_clientes.append((id_cliente, nombre, apellido, mail ))

In [20]:
query_clientes = """
    INSERT INTO clientes (id_cliente, nombre, apellido, mail)
    VALUES (%s, %s, %s, %s)
"""

In [21]:
cur.executemany(query_clientes, d_insert_clientes)
conn.commit()

In [ ]:
df_reservas

In [13]:
df_reservas['id_reserva'] = df_reservas['id_reserva'].str.replace('-', '', regex=False)

In [ ]:
df_reservas['id_hotel'] = df_reservas['id_hotel'].str.extract('(\d+)').astype(int)

In [ ]:
df_reservas.head(30)

In [15]:
d_insert_reservas = []

In [ ]:
hoteles_dict = dict(cur.fetchall())

In [24]:
for i, row in df_reservas.iterrows():
    id_reserva = row['id_reserva']
    fecha_reserva = row['fecha_reserva']
    inicio_estancia = row['inicio_estancia']
    final_estancia = row['final_estancia']
    precio_noche = row['precio_noche']
    id_cliente = row['id_cliente']
    id_hotel = row['id_hotel'] if row['id_hotel'] == 0 else None

    d_insert_reservas.append((id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel))


In [25]:
query_reservas = """
    INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

Da error al hacer la subida porque poir el tipo de dato que se ha de subir se han modificado los ID de hoteles partiendo de 1 y mas tarde me he dado cuenta de que reservas tenia el antiguo ID, al solucionarlo quitando la palabra ID me he dado cuenta de que partia de cero y la base de datos da error.

In [ ]:
cur.executemany(query_reservas, d_insert_reservas)
conn.commit()

In [34]:
cur.close()
conn.close()